In [1]:
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import transformations
import config
%matplotlib inline

In [31]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery('2,7,8,9')", conn)


starter_data = RideWaits.copy()

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\Chrisrtopher\Documents\DisneyWaits\src\transformations.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [97]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop([column_name], axis = 1)
    return df

def get_shift(day, steps):
    previous_steps = {}
    for i in range(1,1+steps):
        current_steps = []
        test_day_current = day.reset_index()
        for index,row in test_day_current.iterrows():
            if index in list(range(i)):
                current_steps.append(0)
            else:
                current_steps.append(test_day_current.loc[index - i,'Wait'])
        
        name = "previous_step"+str(i)
        previous_steps[name] = current_steps
    
    for key,value in previous_steps.items():
        day[key] = value
        
    return day
    
    
def shift_data(ride_data, shift_range):
    new_data_frame = pd.DataFrame()
    distinct_rides = list(ride_data['RideId'].unique())
    for ride in distinct_rides:
        this_ride = ride_data[ride_data['RideId'] == ride]
        day_list = list(this_ride['Date'].unique())
        for day in day_list:
            day_data = this_ride[this_ride['Date'] == day]
            new_data = get_shift(day_data, shift_range)
            new_data_frame = pd.concat([new_data_frame, new_data])

    return new_data_frame

def model_transformation(data, num_shifts):
    ride_waits = data
    
    important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits[ride_waits['Location'] != ""]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    ride_waits = shift_data(ride_waits,num_shifts)
    
    dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
    ride_waits = ride_waits.drop(['Name'], axis = 1)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    correlation = ride_waits.corr()['Wait']
    key_correlations = correlation[abs(correlation) > .075]
    important_cols = list(key_correlations.index)
    shift_columns = ["previous_step" + str(x+1)  for x in range(num_shifts)]
    important_cols = important_cols + ["Wait","MinutesSinceOpen"] + shift_columns
    important_cols = [x for x in important_cols if x != "Weekend_0"]
    important_cols = list(set(important_cols))
    ride_waits_key = ride_waits[important_cols]
    
    
    return ride_waits_key



def new_data_transform(data, num_shifts, important_cols):
    ride_waits = data
    
    important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits[ride_waits['Location'] != ""]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    ride_waits = shift_data(ride_waits,num_shifts)
    
    dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
    ride_waits = ride_waits.drop(['Name'], axis = 1)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    missing_cols = [x for x in important_cols if x not in ride_waits.columns]
    for col in missing_cols:
        ride_waits[col] = 0
        
    return ride_waits

In [6]:
model_data = model_transformation(RideWaits, 3)

C:\Users\Chrisrtopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
model_data.columns

Index(['RideId_18904138', 'RideId_18665186', 'RideId_80010176',
       'Location_Toy Story Land', 'DayOfWeek_5', 'RideId_18665185',
       'previous_step3', 'RideId_18375495', 'IntellectualProp_Avatar',
       'RideId_80010182', 'previous_step2', 'Location_Grand Avenue',
       'IntellectualProp_Fab5', 'Location_Future World', 'RideId_107785',
       'Month_10', 'RideId_209857', 'Location_Animation Courtyard',
       'MinutesSinceOpen', 'Tier_diversion', 'Month_9',
       'Location_Pandora   The World of Avatar', 'IntellectualProp_StarWars',
       'Weekend_1', 'Tier_super_headliner', 'Wait', 'RideId_80010162',
       'TimeSinceMidday', 'Location_Fantasyland', 'Location_Sunset Boulevard',
       'RideId_80010152', 'RideId_80010151', 'ParkId_9', 'Tier_headliner',
       'RideId_80010228', 'RideId_207395', 'Location_Echo Lake', 'ParkId_2',
       'Location_DinoLand U.S.A.', 'RideId_80010190', 'RideId_16767263',
       'RideId_62992', 'Tier_minor_attraction', 'IntellectualProp_Muppets',
 

In [8]:
best_params = {'criterion': 'mse',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

In [10]:
important_columns = [x for x in model_data.columns if x != "Wait"]

In [12]:
%%time
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(**best_params)
clf.fit(model_data[important_columns], model_data['Wait'])


Wall time: 32.8 s


In [13]:
#now we have a model, now we need to create a new data frame for values with the columns listed above for prediction

In [37]:
rides = list(set(starter_data['Name']))
current_ride = starter_data[starter_data['Name'] == rides[0]]

In [112]:
ride_starter = current_ride.iloc[[0]]

In [41]:
from datetime import datetime
from pytz import timezone

In [65]:
tz = timezone('US/Eastern')

In [68]:
dtime = datetime.now(tz)
dtime = dtime.replace(minute = 0, second = 0, microsecond = 0)
date = dtime.date()
time = dtime.time().strftime("%H:%M")

In [69]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [80]:
end_time = dtime.replace(hour = 23, minute = 45, second = 0, microsecond = 0)

In [81]:
time_list = date_range(dtime, end_time, 15, 'minutes')

In [82]:
time_list = [x.time().strftime("%H:%M") for x in time_list]

In [114]:

predictions_frame = pd.DataFrame()
for time in time_list:
    ride_starter['Time'] = time
    row_data = ride_starter.copy()
    try:
        row_data = transformations.transformData(row_data)
    except:
        continue
    print(row_data)
    predictions_frame = pd.concat([predictions_frame,row_data])
    model_predictions_frame = new_data_transform(predictions_frame,3, important_columns)
    predictions_frame['Wait'] = clf.predict(model_predictions_frame[important_columns])
    print(predictions_frame)



C:\Users\Chrisrtopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


     RideId       Date      Time  Wait                                   Name  \
5  80010152 2018-09-19  18:00:00     5  Journey Into Imagination With Figment   

  OpeningDate              Tier      Location IntellectualProp ParkId  \
5  2002-06-01  major_attraction  Future World             None      2   

         ...         CharacterExperience inEMH  validTime  EMHDay  \
5        ...                           0     0          1       0   

  TimeSinceOpen TimeSinceMidday  MagicHourType  MinutesSinceOpen  Month  \
5             9               4           None             540.0      9   

   TimeSinceRideOpen  
5               5954  

[1 rows x 27 columns]
     RideId       Date      Time      Wait  \
5  80010152 2018-09-19  18:00:00  7.579806   

                                    Name OpeningDate              Tier  \
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   

       Location IntellectualProp ParkId        ...          \
5  Future World            

[1 rows x 27 columns]
     RideId       Date      Time       Wait  \
5  80010152 2018-09-19  18:00:00   7.579806   
5  80010152 2018-09-19  18:15:00  10.095982   
5  80010152 2018-09-19  18:30:00  10.095982   
5  80010152 2018-09-19  18:45:00  10.095982   
5  80010152 2018-09-19  19:00:00  10.095982   
5  80010152 2018-09-19  19:15:00  10.095982   

                                    Name OpeningDate              Tier  \
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   

       Location IntellectualProp ParkId        ...          \
5  Future World             None      2        ...           
5  Fu

     RideId       Date      Time       Wait  \
5  80010152 2018-09-19  18:00:00   7.579806   
5  80010152 2018-09-19  18:15:00  10.095982   
5  80010152 2018-09-19  18:30:00  10.095982   
5  80010152 2018-09-19  18:45:00  10.095982   
5  80010152 2018-09-19  19:00:00  10.095982   
5  80010152 2018-09-19  19:15:00  10.095982   
5  80010152 2018-09-19  19:30:00  10.095982   
5  80010152 2018-09-19  19:45:00  10.095982   
5  80010152 2018-09-19  20:00:00  10.095982   
5  80010152 2018-09-19  20:15:00  10.095982   

                                    Name OpeningDate              Tier  \
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figme

     RideId       Date      Time       Wait  \
5  80010152 2018-09-19  18:00:00   7.579806   
5  80010152 2018-09-19  18:15:00  10.095982   
5  80010152 2018-09-19  18:30:00  10.095982   
5  80010152 2018-09-19  18:45:00  10.095982   
5  80010152 2018-09-19  19:00:00  10.095982   
5  80010152 2018-09-19  19:15:00  10.095982   
5  80010152 2018-09-19  19:30:00  10.095982   
5  80010152 2018-09-19  19:45:00  10.095982   
5  80010152 2018-09-19  20:00:00  10.095982   
5  80010152 2018-09-19  20:15:00  10.095982   
5  80010152 2018-09-19  20:30:00  10.095982   
5  80010152 2018-09-19  20:45:00  10.095982   

                                    Name OpeningDate              Tier  \
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Imagination With Figment  2002-06-01  major_attraction   
5  Journey Into Im

In [115]:
predictions_frame

,RideId,Date,Time,Wait,Name,OpeningDate,Tier,Location,IntellectualProp,ParkId,...,CharacterExperience,inEMH,validTime,EMHDay,TimeSinceOpen,TimeSinceMidday,MagicHourType,MinutesSinceOpen,Month,TimeSinceRideOpen
5,80010152,2018-09-19,18:00:00,7.579806,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,9,4,None,540.0,9,5954
5,80010152,2018-09-19,18:15:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,9,4,None,555.0,9,5954
5,80010152,2018-09-19,18:30:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,9,4,None,570.0,9,5954
5,80010152,2018-09-19,18:45:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,9,4,None,585.0,9,5954
5,80010152,2018-09-19,19:00:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,10,5,None,600.0,9,5954
5,80010152,2018-09-19,19:15:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,10,5,None,615.0,9,5954
5,80010152,2018-09-19,19:30:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,10,5,None,630.0,9,5954
5,80010152,2018-09-19,19:45:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,10,5,None,645.0,9,5954
5,80010152,2018-09-19,20:00:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,11,6,None,660.0,9,5954
5,80010152,2018-09-19,20:15:00,10.095982,Journey Into Imagination With Figment,2002-06-01,major_attraction,Future World,None,2,...,0,0,1,0,11,6,None,675.0,9,5954


In [116]:
#perhaps for first of day predictions do not incorporate a shifting feature